## Carregando as bibliotecas

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, expr, upper, asc_nulls_last, desc, lit, when 
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.window import Window
from unidecode import unidecode

## Iniciando o Spark

In [2]:
import findspark
findspark.init('/opt/apache-spark/')

In [3]:
spark = SparkSession.builder.getOrCreate()

21/11/03 01:12:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Carregando os dados

In [4]:
df_titles = spark.read.format('csv').load('base/title_basics.tsv', header=True, sep='\t')

In [5]:
df_titles.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [18]:
df_titles.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



## Expressões

In [21]:
(
    df_titles.select('tconst', 'primaryTitle', 'runtimeMinutes', )
    .withColumn('runtimeHours', round(col('runTimeMinutes').cast('int') / 60, 3))
    .show(5)
)

+---------+--------------------+--------------+------------+
|   tconst|        primaryTitle|runtimeMinutes|runtimeHours|
+---------+--------------------+--------------+------------+
|tt0000001|          Carmencita|             1|       0.017|
|tt0000002|Le clown et ses c...|             5|       0.083|
|tt0000003|      Pauvre Pierrot|             4|       0.067|
|tt0000004|         Un bon bock|            12|         0.2|
|tt0000005|    Blacksmith Scene|             1|       0.017|
+---------+--------------------+--------------+------------+
only showing top 5 rows



In [22]:
(
    df_titles.select('tconst', 'primaryTitle', 'runtimeMinutes', )
    .withColumn('runtimeHours', round(df_titles['runTimeMinutes'].cast('int') / 60, 3))
    .show(5)
)

+---------+--------------------+--------------+------------+
|   tconst|        primaryTitle|runtimeMinutes|runtimeHours|
+---------+--------------------+--------------+------------+
|tt0000001|          Carmencita|             1|       0.017|
|tt0000002|Le clown et ses c...|             5|       0.083|
|tt0000003|      Pauvre Pierrot|             4|       0.067|
|tt0000004|         Un bon bock|            12|         0.2|
|tt0000005|    Blacksmith Scene|             1|       0.017|
+---------+--------------------+--------------+------------+
only showing top 5 rows



In [23]:
(
    df_titles.select('tconst', 'primaryTitle', 'runtimeMinutes', )
    .withColumn('runtimeHours', round(col('runTimeMinutes').cast('int') / 60, 3))
    .withColumn('hours_plus2', col('runtimeHours') + 2)
    .show(5)
)

+---------+--------------------+--------------+------------+-----------+
|   tconst|        primaryTitle|runtimeMinutes|runtimeHours|hours_plus2|
+---------+--------------------+--------------+------------+-----------+
|tt0000001|          Carmencita|             1|       0.017|      2.017|
|tt0000002|Le clown et ses c...|             5|       0.083|      2.083|
|tt0000003|      Pauvre Pierrot|             4|       0.067|      2.067|
|tt0000004|         Un bon bock|            12|         0.2|        2.2|
|tt0000005|    Blacksmith Scene|             1|       0.017|      2.017|
+---------+--------------------+--------------+------------+-----------+
only showing top 5 rows



In [28]:
(
    df_titles.select('tconst', 'primaryTitle', 'runtimeMinutes', )
    .withColumn('runtimeHours', expr('round(cast(runTimeMinutes as INT) / 60, 3)'))
    .show(5)
)

+---------+--------------------+--------------+------------+
|   tconst|        primaryTitle|runtimeMinutes|runtimeHours|
+---------+--------------------+--------------+------------+
|tt0000001|          Carmencita|             1|       0.017|
|tt0000002|Le clown et ses c...|             5|       0.083|
|tt0000003|      Pauvre Pierrot|             4|       0.067|
|tt0000004|         Un bon bock|            12|         0.2|
|tt0000005|    Blacksmith Scene|             1|       0.017|
+---------+--------------------+--------------+------------+
only showing top 5 rows



In [30]:
df_titles.select('tconst', 'genres', upper('primaryTitle').alias('primaryTitle')).show(10)

+---------+--------------------+--------------------+
|   tconst|              genres|        primaryTitle|
+---------+--------------------+--------------------+
|tt0000001|   Documentary,Short|          CARMENCITA|
|tt0000002|     Animation,Short|LE CLOWN ET SES C...|
|tt0000003|Animation,Comedy,...|      PAUVRE PIERROT|
|tt0000004|     Animation,Short|         UN BON BOCK|
|tt0000005|        Comedy,Short|    BLACKSMITH SCENE|
|tt0000006|               Short|   CHINESE OPIUM DEN|
|tt0000007|         Short,Sport|CORBETT AND COURT...|
|tt0000008|   Documentary,Short|EDISON KINETOSCOP...|
|tt0000009|       Romance,Short|          MISS JERRY|
|tt0000010|   Documentary,Short| LEAVING THE FACTORY|
+---------+--------------------+--------------------+
only showing top 10 rows



In [31]:
df_titles.selectExpr('tconst', 'genres', 'upper(primaryTitle) as primaryTitle').show(10)

+---------+--------------------+--------------------+
|   tconst|              genres|        primaryTitle|
+---------+--------------------+--------------------+
|tt0000001|   Documentary,Short|          CARMENCITA|
|tt0000002|     Animation,Short|LE CLOWN ET SES C...|
|tt0000003|Animation,Comedy,...|      PAUVRE PIERROT|
|tt0000004|     Animation,Short|         UN BON BOCK|
|tt0000005|        Comedy,Short|    BLACKSMITH SCENE|
|tt0000006|               Short|   CHINESE OPIUM DEN|
|tt0000007|         Short,Sport|CORBETT AND COURT...|
|tt0000008|   Documentary,Short|EDISON KINETOSCOP...|
|tt0000009|       Romance,Short|          MISS JERRY|
|tt0000010|   Documentary,Short| LEAVING THE FACTORY|
+---------+--------------------+--------------------+
only showing top 10 rows



In [32]:
df_titles.select('startYear').distinct().show()

+---------+
|startYear|
+---------+
|     1903|
|     1953|
|     1897|
|     1957|
|     1987|
|     1956|
|     1936|
|     2016|
|     2020|
|     2012|
|     1958|
|     1910|
|     1943|
|     1915|
|     1972|
|     1931|
|     2026|
|     1911|
|     1926|
|     1938|
+---------+
only showing top 20 rows



## Filtros

In [34]:
(
    df_titles.filter(col('titleType') == 'movie')
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000502|    movie|            Bohemios|            Bohemios|      0|     1905|     \N|           100|                  \N|
|tt0000574|    movie|The Story of the ...|The Story of the ...|      0|     1906|     \N|            70|Action,Adventure,...|
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0000615|    movie|  Robbery Under Arms|  Robbery Under Arms|      0|     1907|     \N|            \N|               Drama|
|tt0000630|    movie|              Hamlet|              Amleto|      0|     1908|     \N|            \N|              

In [35]:
(
    df_titles.where(col('titleType') == 'movie')
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000502|    movie|            Bohemios|            Bohemios|      0|     1905|     \N|           100|                  \N|
|tt0000574|    movie|The Story of the ...|The Story of the ...|      0|     1906|     \N|            70|Action,Adventure,...|
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0000615|    movie|  Robbery Under Arms|  Robbery Under Arms|      0|     1907|     \N|            \N|               Drama|
|tt0000630|    movie|              Hamlet|              Amleto|      0|     1908|     \N|            \N|              

In [37]:
(
    df_titles.filter((col('titleType') == 'movie') & (col('runtimeMinutes') <= 90))
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000574|    movie|The Story of the ...|The Story of the ...|      0|     1906|     \N|            70|Action,Adventure,...|
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Dram

In [38]:
(
    df_titles.filter(((col('titleType') == 'movie') | (col('titleType') == 'tvSeries')) & (col('runtimeMinutes') <= 90))
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000574|    movie|The Story of the ...|The Story of the ...|      0|     1906|     \N|            70|Action,Adventure,...|
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Dram

In [39]:
(
    df_titles.filter((col('titleType').isin('movie','tvSeries')) & (col('runtimeMinutes') <= 90))
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000574|    movie|The Story of the ...|The Story of the ...|      0|     1906|     \N|            70|Action,Adventure,...|
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Dram

In [40]:
(
    df_titles
    .filter(col('titleType').isin('movie','tvSeries'))
    .filter(col('runtimeMinutes') <= 90)
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000574|    movie|The Story of the ...|The Story of the ...|      0|     1906|     \N|            70|Action,Adventure,...|
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Dram

## Filtro com Expressões

In [41]:
(
    df_titles
    .filter('titleType = "movie"')
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000502|    movie|            Bohemios|            Bohemios|      0|     1905|     \N|           100|                  \N|
|tt0000574|    movie|The Story of the ...|The Story of the ...|      0|     1906|     \N|            70|Action,Adventure,...|
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0000615|    movie|  Robbery Under Arms|  Robbery Under Arms|      0|     1907|     \N|            \N|               Drama|
|tt0000630|    movie|              Hamlet|              Amleto|      0|     1908|     \N|            \N|              

In [43]:
(
    df_titles
    .filter('titleType in ("movie", "tvSeries") and runtimeMinutes <= 90')
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000574|    movie|The Story of the ...|The Story of the ...|      0|     1906|     \N|            70|Action,Adventure,...|
|tt0000591|    movie|    The Prodigal Son|   L'enfant prodigue|      0|     1907|     \N|            90|               Drama|
|tt0001184|    movie|Don Juan de Serra...|Don Juan de Serra...|      0|     1910|     \N|            58|     Adventure,Drama|
|tt0001258|    movie|The White Slave T...|Den hvide slaveha...|      0|     1910|     \N|            45|               Drama|
|tt0001285|    movie|   The Life of Moses|   The Life of Moses|      0|     1909|     \N|            50|Biography,Dram

In [45]:
(
    df_titles
    .filter(col('primaryTitle').like('Avengers%'))
    .filter(col('titleType') == 'movie')
    .show()
)

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
| tt0069746|    movie|Avengers of the Reef|Avengers of the Reef|      0|     1973|     \N|            84|    Adventure,Family|
|tt11590024|    movie|          Avengers 5|  Avengers: Blue Sky|      0|     2024|     \N|            \N|                  \N|
|tt13925114|    movie|Avengers: Infinit...|Avengers: Infinit...|      0|     2019|     \N|           135|Action,Adventure,...|
| tt2395427|    movie|Avengers: Age of ...|Avengers: Age of ...|      0|     2015|     \N|           141|Action,Adventure,...|
| tt4154756|    movie|Avengers: Infinit...|Avengers: Infinit...|      0|     2018|     \N|           149|Action

In [46]:
(
    df_titles
    .withColumn('startYear', col('startYear').cast('int'))
    .filter(col('startYear').isNull())
    .show()
)

+---------+------------+--------------------+--------------------+-------+---------+-------+--------------+----------------+
|   tconst|   titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|          genres|
+---------+------------+--------------------+--------------------+-------+---------+-------+--------------+----------------+
|tt0067098|   tvEpisode|         Willi Forst|         Willi Forst|      0|     null|     \N|            55|              \N|
|tt0071158|   tvEpisode|  The Arcata Promise|  The Arcata Promise|      0|     null|     \N|            \N|           Drama|
|tt0073399|       movie|Atlantic City Jac...|           The Money|      0|     null|     \N|            88|    Action,Drama|
|tt0085677|   tvEpisode|        High Country|        High Country|      0|     null|     \N|            \N|           Sport|
|tt0086784|tvMiniSeries|  Predel vozmozhnogo|  Predel vozmozhnogo|      0|     null|     \N|           380|           Drama|


In [48]:
(
    df_titles
    .withColumn('startYear', col('startYear').cast('int'))
    .filter('startYear is null')
    .show()
)

+---------+------------+--------------------+--------------------+-------+---------+-------+--------------+----------------+
|   tconst|   titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|          genres|
+---------+------------+--------------------+--------------------+-------+---------+-------+--------------+----------------+
|tt0067098|   tvEpisode|         Willi Forst|         Willi Forst|      0|     null|     \N|            55|              \N|
|tt0071158|   tvEpisode|  The Arcata Promise|  The Arcata Promise|      0|     null|     \N|            \N|           Drama|
|tt0073399|       movie|Atlantic City Jac...|           The Money|      0|     null|     \N|            88|    Action,Drama|
|tt0085677|   tvEpisode|        High Country|        High Country|      0|     null|     \N|            \N|           Sport|
|tt0086784|tvMiniSeries|  Predel vozmozhnogo|  Predel vozmozhnogo|      0|     null|     \N|           380|           Drama|


## Ordenação

In [55]:
df_titles.orderBy(col('startYear')).filter('titleType = "movie"').show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt2210499|    movie|          Birmingham|          Birmingham|      0|     1896|     \N|            61|         Documentary|
|tt0229676|    movie|Reproduction of t...|Reproduction of t...|      0|     1897|     \N|            \N|Documentary,News,...|
|tt0138349|    movie|O Carnaval em Lisboa|O Carnaval em Lisboa|      0|     1898|     \N|            \N|         Documentary|
|tt0138759|    movie|A Rua Augusta em ...|A Rua Augusta em ...|      0|     1898|     \N|            \N|         Documentary|
|tt0138342|    movie|      O Campo Grande|      O Campo Grande|      0|     1898|     \N|            \N|         Docum

In [56]:
df_titles.orderBy('startYear', desc('runtimeMinutes')).filter('titleType = "movie"').show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt2210499|    movie|          Birmingham|          Birmingham|      0|     1896|     \N|            61|         Documentary|
|tt0229676|    movie|Reproduction of t...|Reproduction of t...|      0|     1897|     \N|            \N|Documentary,News,...|
|tt0138349|    movie|O Carnaval em Lisboa|O Carnaval em Lisboa|      0|     1898|     \N|            \N|         Documentary|
|tt0138342|    movie|      O Campo Grande|      O Campo Grande|      0|     1898|     \N|            \N|         Documentary|
|tt0138759|    movie|A Rua Augusta em ...|A Rua Augusta em ...|      0|     1898|     \N|            \N|         Docum

In [57]:
df_titles.orderBy(col('startYear').cast('int')).show(5)

+---------+---------+-------------+-------------+-------+---------+-------+--------------+-------------+
|   tconst|titleType| primaryTitle|originalTitle|isAdult|startYear|endYear|runtimeMinutes|       genres|
+---------+---------+-------------+-------------+-------+---------+-------+--------------+-------------+
|tt1699743|    movie|   Blind Rage|   Blind Rage|      0|       \N|     \N|            \N|       Action|
|tt1699751|    movie|Old St. Louis|Old St. Louis|      0|       \N|     \N|            \N| Comedy,Drama|
|tt1699744|    movie|  Boilerplate|  Boilerplate|      0|       \N|     \N|            \N|Action,Sci-Fi|
|tt1699749|  tvMovie|  K Blows Top|  K Blows Top|      0|       \N|     \N|            \N|    Biography|
|tt1699750|    movie|      Miranda|      Miranda|      0|       \N|     \N|            \N|        Drama|
+---------+---------+-------------+-------------+-------+---------+-------+--------------+-------------+
only showing top 5 rows



In [60]:
(
    df_titles
    .withColumn('startYear', col('startYear').cast('int'))
    .orderBy(asc_nulls_last('startYear'))
    .show(5)
)

+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|           genres|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+
| tt3155794|    short|    Passage de Venus|    Passage de Venus|      0|     1874|     \N|             1|Documentary,Short|
|tt14495706|    short|   La Rosace Magique|   La Rosace Magique|      0|     1877|     \N|             1|  Animation,Short|
| tt2221420|    short|Sallie Gardner at...|Sallie Gardner at...|      0|     1878|     \N|             1|Documentary,Short|
|tt12592084|    short|   Le singe musicien|   Le singe musicien|      0|     1878|     \N|             1|  Animation,Short|
| tt7816420|    short|Athlete Swinging ...|Athlete Swinging ...|      0|     1881|     \N|             1|Documentary,Short|
+-------

## Renomeando Colunas

In [62]:
(
    df_titles
    .withColumnRenamed('primaryTitle', 'nome_filme')
    .show(5)
)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|          nome_filme|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [65]:
(
    df_titles
    .select(col('primaryTitle').alias('nome_filme'), 'titleType', 'startYear', 'runtimeMinutes')
    .show(5)
)

+--------------------+---------+---------+--------------+
|          nome_filme|titleType|startYear|runtimeMinutes|
+--------------------+---------+---------+--------------+
|          Carmencita|    short|     1894|             1|
|Le clown et ses c...|    short|     1892|             5|
|      Pauvre Pierrot|    short|     1892|             4|
|         Un bon bock|    short|     1892|            12|
|    Blacksmith Scene|    short|     1893|             1|
+--------------------+---------+---------+--------------+
only showing top 5 rows



In [66]:
(
    df_titles
    .selectExpr('primaryTitle as nome_filme', 'titleType', 'startYear', 'runtimeMinutes')
    .show(5)
)

+--------------------+---------+---------+--------------+
|          nome_filme|titleType|startYear|runtimeMinutes|
+--------------------+---------+---------+--------------+
|          Carmencita|    short|     1894|             1|
|Le clown et ses c...|    short|     1892|             5|
|      Pauvre Pierrot|    short|     1892|             4|
|         Un bon bock|    short|     1892|            12|
|    Blacksmith Scene|    short|     1893|             1|
+--------------------+---------+---------+--------------+
only showing top 5 rows



## Criando e Alaterando Colunas

In [67]:
(
    df_titles.select('tconst', 'primaryTitle', 'runtimeMinutes', )
    .withColumn('runtimeHours', round(col('runTimeMinutes').cast('int') / 60, 3))
    .show(5)
)

+---------+--------------------+--------------+------------+
|   tconst|        primaryTitle|runtimeMinutes|runtimeHours|
+---------+--------------------+--------------+------------+
|tt0000001|          Carmencita|             1|       0.017|
|tt0000002|Le clown et ses c...|             5|       0.083|
|tt0000003|      Pauvre Pierrot|             4|       0.067|
|tt0000004|         Un bon bock|            12|         0.2|
|tt0000005|    Blacksmith Scene|             1|       0.017|
+---------+--------------------+--------------+------------+
only showing top 5 rows



## Criando colunas a partir de constantes

In [70]:
(
    df_titles
    .select('tconst', 'primaryTitle', 'runtimeMinutes')
    .withColumn('pais', lit('Brasil'))
    .show(5)
)

+---------+--------------------+--------------+------+
|   tconst|        primaryTitle|runtimeMinutes|  pais|
+---------+--------------------+--------------+------+
|tt0000001|          Carmencita|             1|Brasil|
|tt0000002|Le clown et ses c...|             5|Brasil|
|tt0000003|      Pauvre Pierrot|             4|Brasil|
|tt0000004|         Un bon bock|            12|Brasil|
|tt0000005|    Blacksmith Scene|             1|Brasil|
+---------+--------------------+--------------+------+
only showing top 5 rows



In [75]:
(
    df_titles
    .select('tconst', 'primaryTitle', 'runtimeMinutes')
    .withColumn('runtimeMinutes', col('runtimeMinutes').cast('int'))
    .withColumn('categoria_runtime', when(col('runtimeMinutes') <= 60, 'curto')
                                     .when((col('runtimeMinutes') > 60) & (col('runtimeMinutes') < 120), 'normal')
                                     .when(col('runtimeMinutes') >= 120, 'longo')
                                     .when(col('runtimeMinutes') .isNull(), 'nulo')
                                     .otherwise('Erro'))
    .filter('runtimeMinutes > 60')
    .show(25)
)

+---------+--------------------+--------------+-----------------+
|   tconst|        primaryTitle|runtimeMinutes|categoria_runtime|
+---------+--------------------+--------------+-----------------+
|tt0000502|            Bohemios|           100|           normal|
|tt0000574|The Story of the ...|            70|           normal|
|tt0000591|    The Prodigal Son|            90|           normal|
|tt0000679|The Fairylogue an...|           120|            longo|
|tt0001756|Lucha por la here...|            92|           normal|
|tt0002026|Anny - Story of a...|            68|           normal|
|tt0002101|           Cleopatra|           100|           normal|
|tt0002130|     Dante's Inferno|            71|           normal|
|tt0002315|El lobo de la sierra|            76|           normal|
|tt0002423|             Passion|            85|           normal|
|tt0002445|          Quo Vadis?|           120|            longo|
|tt0002452|The Independence ...|           120|            longo|
|tt0002625

In [77]:
predicado = """
CASE WHEN runtimeMinutes <= 60 THEN 'curto'
     WHEN runtimeMinutes > 60 AND runtimeMinutes < 120 THEN 'normal'
     WHEN runtimeMinutes >= 120 THEN 'longo'
     WHEN runtimeMinutes IS NUll THEN 'nulo'
ELSE 'erro'
END
"""

(
    df_titles
    .select('tconst', 'primaryTitle', 'runtimeMinutes')
    .withColumn('runtimeMinutes', col('runtimeMinutes').cast('int'))
    .withColumn('categoria_runtime', expr(predicado))
    .filter('runtimeMinutes > 60')
    .show(25)
)

+---------+--------------------+--------------+-----------------+
|   tconst|        primaryTitle|runtimeMinutes|categoria_runtime|
+---------+--------------------+--------------+-----------------+
|tt0000502|            Bohemios|           100|           normal|
|tt0000574|The Story of the ...|            70|           normal|
|tt0000591|    The Prodigal Son|            90|           normal|
|tt0000679|The Fairylogue an...|           120|            longo|
|tt0001756|Lucha por la here...|            92|           normal|
|tt0002026|Anny - Story of a...|            68|           normal|
|tt0002101|           Cleopatra|           100|           normal|
|tt0002130|     Dante's Inferno|            71|           normal|
|tt0002315|El lobo de la sierra|            76|           normal|
|tt0002423|             Passion|            85|           normal|
|tt0002445|          Quo Vadis?|           120|            longo|
|tt0002452|The Independence ...|           120|            longo|
|tt0002625

## Agragação e Agrupamento

In [6]:
df_title_subset = (
    df_titles
    .filter("cast(startYear as int) >= 2000")
    .sample(fraction = 0.5)
    .withColumn('genre', f.split('genres', ',').getItem(0))
)

In [7]:
(
    df_title_subset
    .agg(f.expr('count(distinct genre) as distinct_genre'))
    .show(5)
)

+--------------+
|distinct_genre|
+--------------+
|            28|
+--------------+



In [110]:
(
    df_title_subset
    .withColumn('runtimeMinutes', f.col('runtimeMinutes').cast('int'))
    .agg(f.sum('runtimeMinutes').alias('total_runtimeMinutes'),
         f.mean('runtimeMinutes').alias('mean_runtimeMinutes'),
         f.min('runtimeMinutes').alias('min_runtimeMinutes'),
         f.max('runtimeMinutes').alias('max_runtimeMinutes'),
         f.percentile_approx('runtimeMinutes', 0.5, f.lit(10000000)).alias('median_runtimeMinutes'),
         f.stddev('runtimeMinutes').alias('std_runtimeMinutes'),)
    .toPandas()
)

,total_runtimeMinutes,mean_runtimeMinutes,min_runtimeMinutes,max_runtimeMinutes,median_runtimeMinutes,std_runtimeMinutes
0,33611661,40.800754,0,51420,26,88.770505


In [113]:
df_title_subset.select('runtimeMinutes').describe().toPandas()

,summary,runtimeMinutes
0,count,2703972
1,mean,40.80075382374363
2,stddev,88.77050453432403
3,min,0
4,max,\N


## Agrupamento

In [115]:
(
    df_title_subset
    .groupBy('genre')
    .count()
    .orderBy(f.col('count').desc())
    .toPandas()
)

,genre,count
0,Comedy,478498
1,Drama,449019
2,Documentary,238025
3,\N,184158
4,Talk-Show,181948
5,News,161666
6,Reality-TV,119977
7,Action,112245
8,Adult,107854
9,Animation,85221


In [118]:
(
    df_title_subset
    .groupBy('genre', 'startYear')
    .agg(f.mean('runtimeMinutes').alias('mean_runtimeMinutes'))
    .orderBy('startYear', f.col('mean_runtimeMinutes').desc())
    .filter('startYear = 2021')
    .toPandas()
)

,genre,startYear,mean_runtimeMinutes
0,Sport,2021,113.408889
1,War,2021,91.750000
2,Western,2021,90.800000
3,Thriller,2021,88.278107
4,Adventure,2021,74.026258
5,Biography,2021,70.874494
6,\N,2021,70.789216
7,News,2021,61.903976
8,Adult,2021,61.288747
9,Game-Show,2021,55.158009


In [120]:
(
    df_title_subset
    .groupBy('genre')
    .agg(f.collect_list(f.col('titleType')).alias('lista_tipos_titulo'),
         f.count(f.lit(1)).alias('count'))
    .withColumn('n', f.size(f.col('lista_tipos_titulo')))
    .orderBy('genre')
    .toPandas()
)

,genre,lista_tipos_titulo,count,n
0,Action,"[movie, movie, movie, movie, video, movie, mov...",112245,112245
1,Adult,"[video, video, video, video, video, video, vid...",107854,107854
2,Adventure,"[tvMovie, movie, movie, movie, movie, movie, t...",59767,59767
3,Animation,"[short, short, movie, movie, short, movie, mov...",85221,85221
4,Biography,"[tvMovie, movie, movie, movie, movie, tvEpisod...",30182,30182
5,Comedy,"[movie, short, tvMovie, movie, movie, movie, m...",478498,478498
6,Crime,"[tvEpisode, movie, movie, movie, movie, movie,...",78496,78496
7,Documentary,"[short, movie, movie, short, movie, movie, mov...",238025,238025
8,Drama,"[movie, movie, movie, movie, short, movie, mov...",449019,449019
9,Family,"[movie, tvMovie, tvSeries, movie, short, tvMov...",58665,58665


In [ ]:
(
    df_title_subset
    .groupBy('genre')
    .agg(f.collect_list(f.col('titleType')).alias('lista_tipos_titulo'),
         f.expr('count(distinct titleType) as n_distinct'))
    .withColumn('tipo_filme', f.explode(f.col('lista_tipos_titulo')))
    .orderBy('genre')
    .toPandas()
)

## Pivot

In [10]:
(
    df_title_subset
    .drop('genre')
    .withColumn('genres', f.explode(f.split(f.col('genres'), ',')))
    .groupBy('tconst', 'primaryTitle')
    .pivot('genres')
    .count()
    .na.fill(0)
    .orderBy('tconst')
    .limit(5)
    .toPandas()
)

21/11/02 23:01:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
21/11/02 23:01:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
21/11/02 23:01:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
21/11/02 23:01:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
21/11/02 23:01:31 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
21/11/02 23:01:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
21/11/02 23:01:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
21/11/02 23:01:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
21/11/02 23:01:34 WARN RowBasedKeyValueBatch: Calling spill() on

,tconst,primaryTitle,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,\N
0,tt0011216,Spanish Fiesta,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tt0015414,La tierra de los toros,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,tt0018295,El puño de hierro,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0034413,Youth Gets a Break,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,tt0035423,Kate & Leopold,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


## Window Functions

In [13]:
w = Window.partitionBy('genre').orderBy(f.desc('startYear'))
(
    df_title_subset
    .withColumn('genre', f.split('genres', ',').getItem(0))
    .withColumn('startYear', f.col('startYear').cast('int'))
    .withColumn('rn', f.row_number().over(w))
    .limit(5)
    .toPandas()
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genre,rn
0,tt10827306,movie,Ghost Swim,Ghost Swim,0,2023,\N,\N,"Crime,Horror,Mystery",Crime,1
1,tt12238522,tvEpisode,The Interviews,The Interviews,0,2023,\N,\N,Crime,Crime,2
2,tt12275106,tvEpisode,The Websters Division,The Websters Division,0,2023,\N,\N,Crime,Crime,3
3,tt13391780,movie,Spiralled,Spiralled,0,2023,\N,\N,"Crime,Drama",Crime,4
4,tt13703326,tvEpisode,The Demon Walker,The Demon Walker,0,2023,\N,\N,Crime,Crime,5


In [15]:
w = Window.partitionBy('titleType','startYear').orderBy('titleType','startYear')
(
    df_title_subset
    .withColumn('genre', f.split('genres', ',').getItem(0))
    .withColumn('runtimeMinutes', f.col('runtimeMinutes').cast('int'))
    .withColumn('lag_runtime', f.lag('runtimeMinutes').over(w))
    .limit(5)
    .toPandas()
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genre,lag_runtime
0,tt10005776,tvMovie,Nasjonalparken - fra soloppgang til solnedgang,Nasjonalparken - fra soloppgang til solnedgang,0,2014,\N,124,Documentary,Documentary,NaN
1,tt10016288,tvMovie,Eidsvollsbygningen tilbake til 1814,Eidsvollsbygningen tilbake til 1814,0,2014,\N,29,Documentary,Documentary,124.0
2,tt10023298,tvMovie,Gjenerobringen av norske hav,Gjenerobringen av norske hav,0,2014,\N,48,Documentary,Documentary,29.0
3,tt10039018,tvMovie,Pam til Paris,Pam til Paris,0,2014,\N,29,Documentary,Documentary,48.0
4,tt10080018,tvMovie,Hindsight,Hindsight,0,2014,\N,24,Comedy,Comedy,29.0


In [14]:
w = Window.partitionBy('titleType','startYear')
(
    df_title_subset
    .withColumn('genre', f.split('genres', ',').getItem(0))
    .withColumn('runtimeMinutes', f.col('runtimeMinutes').cast('int'))
    .withColumn('total_minutes', f.sum(f.col('runtimeMinutes')).over(w))
    .withColumn('mean_minutes', f.mean(f.col('runtimeMinutes')).over(w))
    .withColumn('relative_minutes', f.col('runtimeMinutes') / f.col('total_minutes'))
    .limit(5)
    .toPandas()
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genre,total_minutes,mean_minutes,relative_minutes
0,tt10005776,tvMovie,Nasjonalparken - fra soloppgang til solnedgang,Nasjonalparken - fra soloppgang til solnedgang,0,2014,\N,124,Documentary,Documentary,67659,69.679712,0.001833
1,tt10016288,tvMovie,Eidsvollsbygningen tilbake til 1814,Eidsvollsbygningen tilbake til 1814,0,2014,\N,29,Documentary,Documentary,67659,69.679712,0.000429
2,tt10023298,tvMovie,Gjenerobringen av norske hav,Gjenerobringen av norske hav,0,2014,\N,48,Documentary,Documentary,67659,69.679712,0.000709
3,tt10039018,tvMovie,Pam til Paris,Pam til Paris,0,2014,\N,29,Documentary,Documentary,67659,69.679712,0.000429
4,tt10080018,tvMovie,Hindsight,Hindsight,0,2014,\N,24,Comedy,Comedy,67659,69.679712,0.000355


In [16]:
w = Window.partitionBy('genre')
(
    df_title_subset
    .withColumn('max_data', f.max(f.col('startYear')).over(w))
    .filter('startYear = max_data')
    .limit(5)
    .toPandas()
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genre,max_data
0,tt10827306,movie,Ghost Swim,Ghost Swim,0,2023,\N,\N,"Crime,Horror,Mystery",Crime,2023
1,tt12238522,tvEpisode,The Interviews,The Interviews,0,2023,\N,\N,Crime,Crime,2023
2,tt12275106,tvEpisode,The Websters Division,The Websters Division,0,2023,\N,\N,Crime,Crime,2023
3,tt13391780,movie,Spiralled,Spiralled,0,2023,\N,\N,"Crime,Drama",Crime,2023
4,tt13703326,tvEpisode,The Demon Walker,The Demon Walker,0,2023,\N,\N,Crime,Crime,2023


## Joins

In [5]:
df_ratings = spark.read.format('csv').load('base/title_ratings.tsv', header=True, sep='\t')

In [19]:
df_ratings.limit(5).toPandas()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1809
1,tt0000002,6.0,233
2,tt0000003,6.5,1560
3,tt0000004,6.1,152
4,tt0000005,6.2,2383


In [21]:
(
    df_titles
    .join(df_ratings, 'tconst')
    .limit(5)
    .toPandas()
)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000658,short,The Puppet's Nightmare,Le cauchemar de Fantoche,0,1908,\N,2,"Animation,Short",6.5,212
1,tt0001732,short,The Lighthouse Keeper,The Lighthouse Keeper,0,1911,\N,\N,"Drama,Short",7.1,15
2,tt0002253,short,Home Folks,Home Folks,0,1912,\N,17,"Drama,Short",4.0,13
3,tt0002473,short,The Sands of Dee,The Sands of Dee,0,1912,\N,17,"Romance,Short",6.5,77
4,tt0002588,movie,Zigomar contre Nick Carter,Zigomar contre Nick Carter,0,1912,\N,48,"Crime,Thriller",6.0,32


## Semi join e anti join

In [22]:
(
    df_titles
    .join(df_ratings, 'tconst', 'semi')
    .count()
)

1182639

In [23]:
(
    df_titles
    .join(df_ratings, 'tconst', 'anti')
    .count()
)

7021051

## UNION

In [24]:
df1 = df_titles.sample(fraction=0.5)
df2 = df_titles.join(df1, 'tconst', 'anti')

In [29]:
df_titles.count()

8203690

In [26]:
df1.count()

4101774

In [27]:
df2.count()

4101916

In [28]:
df1.union(df2).count()

8203690

In [30]:
df3 = df_titles.sample(fraction=0.05)

In [31]:
df3.count()

409356

In [32]:
df3.union(df3).distinct().count()

409356

In [33]:
df4 = df2.select(df2.columns[::-1])

In [34]:
df4.show(5)

+-------------------+--------------+-------+---------+-------+--------------------+--------------------+---------+---------+
|             genres|runtimeMinutes|endYear|startYear|isAdult|       originalTitle|        primaryTitle|titleType|   tconst|
+-------------------+--------------+-------+---------+-------+--------------------+--------------------+---------+---------+
|        Drama,Short|            \N|     \N|     1909|      0|  The Curse of Money|  The Curse of Money|    short|tt0000839|
|      Short,Western|            \N|     \N|     1910|      0|A Cowboy's Vindic...|A Cowboy's Vindic...|    short|tt0001170|
|Drama,Romance,Short|            \N|     \N|     1911|      0|        Her Two Sons|        Her Two Sons|    short|tt0001664|
|        Drama,Short|            \N|     \N|     1911|      0|The Lighthouse Ke...|The Lighthouse Ke...|    short|tt0001732|
|       Comedy,Short|            \N|     \N|     1911|      0|     The Skating Bug|     The Skating Bug|    short|tt0001887|


In [35]:
df1.unionByName(df4).limit(5).toPandas()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
1,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
2,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
3,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
4,tt0000011,short,Akrobatisches Potpourri,Akrobatisches Potpourri,0,1895,\N,1,"Documentary,Short"


## UDFs

In [42]:
unidecode('àáâçéõü')

'aaaceou'

In [44]:
def unicode_function(string):
    if not string:
        return None
    else:
        return unidecode(string)
    
unicode_udf = f.udf(unicode_function, returnType=StringType())

In [45]:
(
    df_titles
    .filter(f.col('primaryTitle').rlike('à|á|â|ç|é|õ|ü'))
    .withColumn('cleaned_string', unicode_udf(f.col('primaryTitle')))
    .select('primaryTitle', 'cleaned_string')
    .limit(5)
    .toPandas()
)

,primaryTitle,cleaned_string
0,Arrivée d'un train gare de Vincennes,Arrivee d'un train gare de Vincennes
1,Batteuse à vapeur,Batteuse a vapeur
2,Bébé et fillettes,Bebe et fillettes
3,A Chegada do Comboio Inaugural à Estação Centr...,A Chegada do Comboio Inaugural a Estacao Centr...
4,Cortège de tzar allant à Versailles,Cortege de tzar allant a Versailles


Traceback (most recent call last):
  File "/opt/apache-spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/apache-spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/apache-spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/apache-spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [46]:
del unicode_udf

In [48]:
@f.udf(returnType=StringType())
def unicode_udf(string):
    if not string:
        return None
    else:
        return unidecode(string)

In [49]:
(
    df_titles
    .filter(f.col('primaryTitle').rlike('à|á|â|ç|é|õ|ü'))
    .withColumn('cleaned_string', unicode_udf(f.col('primaryTitle')))
    .select('primaryTitle', 'cleaned_string')
    .limit(5)
    .toPandas()
)

,primaryTitle,cleaned_string
0,Arrivée d'un train gare de Vincennes,Arrivee d'un train gare de Vincennes
1,Batteuse à vapeur,Batteuse a vapeur
2,Bébé et fillettes,Bebe et fillettes
3,A Chegada do Comboio Inaugural à Estação Centr...,A Chegada do Comboio Inaugural a Estacao Centr...
4,Cortège de tzar allant à Versailles,Cortege de tzar allant a Versailles


Traceback (most recent call last):
  File "/opt/apache-spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/opt/apache-spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/opt/apache-spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/opt/apache-spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


## criando metodos customizados

In [50]:
def renamer(dict):
    def _(df):
        for c, n in dict.items():
            df = df.withColumnRenamed(c, n)
        return df
    return _

In [51]:
df_titles.limit(5).toPandas()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [52]:
rename_dict = {
    'tconst' : 'id_title',
    'titleType' : 'tipo_title',
    'primaryTitle': 'nome_primario',
    'originalTitle': 'nome_original',
    'isAdult': 'idc_adult_title',
    'startYear': 'ano_lancamento',
    'endYear': 'ano_encerramento',
    'runtimeMinutes': 'duracao_minutos',
    'genres': 'genero'
}

In [53]:
(
    df_titles
    .transform(renamer(rename_dict))
    .limit(5)
    .toPandas()
)

,id_title,tipo_title,nome_primario,nome_original,idc_adult_title,ano_lancamento,ano_encerramento,duracao_minutos,genero
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


## Trabalho 

In [6]:
df_filmes = (
    df_titles
    .join(df_ratings, 'tconst')
)

In [57]:
df_filmes.limit(5).toPandas()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000658,short,The Puppet's Nightmare,Le cauchemar de Fantoche,0,1908,\N,2,"Animation,Short",6.5,212
1,tt0001732,short,The Lighthouse Keeper,The Lighthouse Keeper,0,1911,\N,\N,"Drama,Short",7.1,15
2,tt0002253,short,Home Folks,Home Folks,0,1912,\N,17,"Drama,Short",4.0,13
3,tt0002473,short,The Sands of Dee,The Sands of Dee,0,1912,\N,17,"Romance,Short",6.5,77
4,tt0002588,movie,Zigomar contre Nick Carter,Zigomar contre Nick Carter,0,1912,\N,48,"Crime,Thriller",6.0,32


In [58]:
df_filmes.select('titleType').distinct().toPandas()

,titleType
0,tvSeries
1,tvMiniSeries
2,tvMovie
3,tvEpisode
4,movie
5,tvSpecial
6,video
7,videoGame
8,tvShort
9,short


In [60]:
(
    df_titles
    .filter('startYear = 2015')
    .filter("titleType in ('movie', 'tvMovie')")
    .count()
)

19987

In [66]:
(
    df_titles
    .withColumn('genres', f.explode(f.split(f.col('genres'), ',')))
    .groupBy('genres')
    .agg(f.expr('count(genres) as qtd'))
    .orderBy(f.expr('count(genres) as qtd'))
    .toPandas()
)

,genres,qtd
0,Film-Noir,763
1,Western,27912
2,War,29827
3,Musical,63122
4,Biography,87738
5,Sci-Fi,96515
6,History,114975
7,Thriller,134054
8,Horror,146400
9,Mystery,162448


In [68]:
(
    df_filmes
    .withColumn('genres', f.explode(f.split(f.col('genres'), ',')))
    .groupBy('genres')
    .agg(f.expr('avg(averageRating) as nota'))
    .orderBy(f.expr('avg(averageRating) as nota'))
    .toPandas()
)

,genres,nota
0,Horror,6.079672
1,Thriller,6.335533
2,Adult,6.349891
3,\N,6.403648
4,Film-Noir,6.552887
5,News,6.591745
6,Musical,6.610591
7,Talk-Show,6.715781
8,Sci-Fi,6.754274
9,Short,6.791292


In [19]:
(
    df_filmes
    .filter("titleType = 'videoGame' ")
    .filter('startYear = 2020')
    .groupBy('primaryTitle')
    .agg(f.expr('max(averageRating)'))
    .orderBy(f.expr('max(averageRating)'))
    .toPandas()
)

,primaryTitle,max(averageRating)
0,Shadow the Ronin,1.9
1,Fast and Furious Crossroads,2.3
2,Road Bustle,2.4
3,XIII Remake,2.4
4,Madden NFL 21,2.7
...,...,...
245,Final Fantasy VII Remake,9.1
246,Ori and the Will of the Wisps,9.1
247,Omori,9.2
248,Ghost of Tsushima,9.3


In [ ]:
predicado = """
CASE WHEN runtimeMinutes <= 60 THEN 'curto'
     WHEN runtimeMinutes > 60 AND runtimeMinutes < 120 THEN 'normal'
     WHEN runtimeMinutes >= 120 THEN 'longo'
     WHEN runtimeMinutes IS NUll THEN 'nulo'
ELSE 'erro'
END
"""

(
    df_titles
    .select('tconst', 'primaryTitle', 'runtimeMinutes')
    .withColumn('runtimeMinutes', col('runtimeMinutes').cast('int'))
    .withColumn('categoria_runtime', expr(predicado))
    .filter('runtimeMinutes > 60')
    .show(25)
)

In [16]:
(
    df_titles
    .filter('startYear = 2018')
    .withColumn('genres', f.split(f.col('genres'), ','))
    .withColumn('comedy_2018', f.when(f.array_contains(f.col('genres'), 'Comedy'), 1).otherwise(0))
    .select((f.sum('comedy_2018') / f.count(f.col('comedy_2018'))) * 100)
    .toPandas()
)

,((sum(comedy_2018) / count(comedy_2018)) * 100)
0,19.592337


In [21]:
def sqr_divide(value): 

    return (value**2)/2 

sqr_divide_udf = f.udf(sqr_divide, DoubleType()) 

In [22]:
(
    df_ratings
    .withColumn('averageRating', f.col('averageRating').cast('double'))
    .select(sqr_divide_udf('averageRating').alias('averageRating'))
    .agg(f.mean('averageRating').alias('averageRating'))
    .show()
)

+------------------+
|     averageRating|
+------------------+
|24.899137999843724|
+------------------+

